# Notes from Survival Analysis Intuition & Implementation in Python Article

Article: [Survival Analysis: Intuition & Implementation in Python](https://towardsdatascience.com/survival-analysis-intuition-implementation-in-python-504fde4fcf8e)

# Intro

Goal is to answer business questions like

- how long will customer remain with business (after how much time will customer churn)
- how long will machine last
- relative retention rate of different marketing channels
- sports drafting, how long will player stay at top level, different draft channels
- likelihood a patient will survive after being diagnosed
- how long would it be before a particular event occurs/time to event analysis

# Definitions

Survival Analysis is also known as Time to Event analysis. We define *time* and *event* in the context of various use cases that may arise. This is pretty straightforward and extendable to other use cases.

|Description | Detail | Event | Time of origin | Time Scale |
| --- | --- | --- | --- | --- | 
|Predictive maintenance in mechanical operations | How long will machine last? | time at which the machine breaks down | is the time of start of machine for the continuous operations | weeks, days, hours
|Customer Analytics | customer retention, churn prevention, customer lifetime value | time at which customer churns/unsubscribes | time at which the customer starts service/subscription with company | months, weeks |
|Marketing Analytics | Cohort Analysis, retention rates of each marketing channel | time at which customer unsubscribes a marketing channel | time at which customer starts service/marketing channel | months, weeks |
|Actuaries | probability of population to die in particular time range (insurance premiums) | death | birth | years |

Difference between time of event and time of origin gives time to event.

# Math

Assume non-negative continuous random variable $T$, the time until an event of interest (e.g. time from start of subscription to churn). We characterize the random variable via its probability density function (PDF) and its cumulative distribution function (CDF). Say the PDF of $T$ is $f(t)$ and the CDF of $T$ is $F(t)$.

Recall that the **CDF** is defined as $F(t) = P(T<t)$, i.e. $F(t)$ is the probability that the event occurred by time $t$. $F(t)$ gives the proportion of population with the time to event value of less than $t$. Given the PDF $f(t)$ we get the CDF via

$$ F(t) = \int_0^t f(x) dx$$

We define the **survival function** $S(t)$ as $S(t) = 1 - F(t) = P(T\geq t)$. $S(t)$ gives the probability that the event has not occurred by the time $t$ (e.g. $S(t)$ gives us the proportion of population with the time to event value more than $t$). Given the PDF $f(t)$ we can get $S(t)$ via 

$$ S(t) = \int_0^{\infty} f(x) dx$$

We define the **hazard funtion** as the rate at which the event is taking place out of surviving population at any given time $t$. "Out of the people who survived at time $t$, what is the rate of dying of those people?". This takes the form

$$ h(t) = \lim_{dt \to 0} \frac{\frac{(S(t) - S(t + dt))}{dt}}{S(t)} $$

We can see there are two parts of this formula. 

First is the instantaneous rate of event:

$$ \frac{S(t) - S(t + dt)}{dt}$$

this can be seen as the slope at any point $t$ of the **survival curve** (the rate of dying at time $t$). Assume the total population is $P$. Observe that $S(t) - S(t + dt)$ is the proportion of people who died in time $dt$ out of the people who had survived at time $t$. The number of people who survived at time $t$ is $S(t)*P$, and the number of people who survived at time $t+dt$ is $S(t+dt)*P$, so the number of people who died during time $dt$ is $(S(t) - S(t + dt))*P$. Thus the instaneous rate of people dying at time $t$ is 

$$ \frac{(S(t) - S(t + dt))*P}{dt}$$

Next is the proportion surviving at time $t$. At time $t$ we know the surviving population is $S(t)*P$. Thus if we can divide the number of people who died in time $dt$ by the number of people survived at any time $t$, we get the hazard function which can serve as a measure of RISK of people dying out of people who had survived at time $t$.

The hazard function is not a density or probability. Instead we can think of it as the probability of failure in an infintesimally small time period between $t$ and $t+dt$, given that the subject has survived up until time $t$. Along this thinking, *hazard is a measure of risk*. The greater the hazard between times $t_1$ and $t_2$, the greature the risk of failure in this time interval. 

Given that (not sure about the $\lim$ below)

$$ f(t) =\lim_{dt\to 0} \frac{S(t) - S(t+dt)}{dt} $$

we know that 

$$h(t) = \frac{f(t)}{S(t)}$$

This is an important derivation. Whats cool is that survival function can be derived from hazard function and vice versa. This becomes useful while deriving a survival function given a hazard function in the Cox Proportional Model (covered below).

These were the important mathematical definitions and the formulations required to understand the survival analysis. The math ends here and we move on to estimating of survival curves below.

### Kaplan-Meier Estimate

In the math above, we assumed the pdf and derived the survival function with it. When we don't have a true survival curve of the population, we can estimate it from the data. Two methods of estimating survival curve are 

- parametric approach, assume a parametric model based on a distribution and estimated parameters
- non-parametric approach called the *Kaplan-Meier Estimator*

$$\hat{S}(t) = \prod_{i:t_i \leq t} \frac{n_i - d}{n_i}$$

Here $n_i$ is the population at risk at time just prior to time $t_i$, $d_i$ is the number of events that occurred at time $t_i$.